In [114]:
import os

os.environ['VASP_PP_PATH']     = '/home/igoratoms/Programs/vasp-6.5.1/pp'
os.environ['ASE_VASP_COMMAND'] = 'mpirun -np 1 vasp_std'
os.environ['NO_STOP_MESSAGE']  = '1' # to avoid warning from mpirun

from ase.calculators.vasp import Vasp
from ase import Atoms
from ase.io import write, read

from ase.visualize.plot import plot_atoms
from ase.visualize import view

from ase.thermochemistry import IdealGasThermo

from ase.lattice.cubic import FaceCenteredCubic
from ase.build import fcc111
from ase.constraints import FixAtoms
from ase.build import add_adsorbate

import numpy as np

### $\text{CO}_{\text{(g)}}$: Relaxation

In [115]:
CO = Atoms('CO',
            positions=[[0.000, 0.000, 0.000],
                       [0.000, 0.000, 1.128]])

CO.center(vacuum=6.0)  
CO.pbc = True

calc_relax = Vasp(

    directory = 'COrelax',
    xc        = 'PBE',
    encut     = 450,
    ismear    = 0,
    sigma     = 0.05,
    ediff     = 1E-6, 
    ediffg    = -0.01,
    isif      = 2, 
    ibrion    = 2,
    nsw       = 100,
    nelm      = 100,
    lwave     = True,
    lcharg    = True,
    lvtot     = True,
    atoms     = CO

)   
calc_relax.calculate(CO)

E_CO = CO.get_potential_energy()

print(f'Optimized energy of CO: {E_CO:.3f} eV')

view(CO, viewer='x3d')

Optimized energy of CO: -14.780 eV


### $\text{CO}_{\text{(g)}}$: Vibration

In [116]:
calc_vib = Vasp(
    directory = 'COvib',
    xc        = 'PBE',
    encut     = 450,
    ismear    = 0,
    sigma     = 0.05,
    ediff     = 1e-8,
    ibrion    = 6,           # finite differences with symmetry
    lreal     = False,
    lwave     = False, 
    lcharg    = False,
    atoms     = CO
)

calc_vib.calculate(CO)

COvib = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = COvib.get_frequencies()

print(f'Vibrational frequencies of CO (cm⁻¹):')
vib_energies_co = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energies_co.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

Vibrational frequencies of CO (cm⁻¹):
f_v: 2125.283 cm⁻¹, E_v: 0.263 eV


### $\text{OH}_{\text{(g)}}$: Relaxation

In [117]:
OH = Atoms('OH',
            positions=[[0.000, 0.000, 0.000],
                       [0.000, 0.000, 0.940]])

OH.center(vacuum=6.0)  
OH.pbc = True

calc_relax = Vasp(

    directory = 'OHrelax',
    xc        = 'PBE',
    encut     = 450,
    ismear    = 0,
    sigma     = 0.05,
    ediff     = 1E-6, 
    ediffg    = -0.01,
    isif      = 2, 
    ibrion    = 2,
    nsw       = 100,
    nelm      = 100,
    lwave     = True,
    lcharg    = True,
    lvtot     = True,
    atoms     = OH

)   
calc_relax.calculate(OH)

E_OH = OH.get_potential_energy()

print(f'Optimized energy of OH: {E_OH:.3f} eV')

view(OH, viewer='x3d')

Optimized energy of OH: -7.096 eV


### $\text{OH}_{\text{(g)}}$: Vibration


In [118]:
calc_vib = Vasp(
    directory = 'OHvib',
    xc        = 'PBE',
    encut     = 450,
    ismear    = 0,
    sigma     = 0.05,
    ediff     = 1e-8,
    ibrion    = 6,           # finite differences with symmetry
    lreal     = False,
    lwave     = False, 
    lcharg    = False,
    atoms     = OH
)

calc_vib.calculate(OH)

COvib = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = COvib.get_frequencies()

print(f'Vibrational frequencies of OH (cm⁻¹):')
vib_energies_oh = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energies_oh.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

Vibrational frequencies of OH (cm⁻¹):
f_v: 3621.570 cm⁻¹, E_v: 0.449 eV


### $\text{cis-COOH}_{\text{(g)}}$: Relaxation

In [119]:
cisCOOH = Atoms('COOH',
    positions=[
        [-0.421859,    0.003009,    0.018584],   # C
        [ 0.773521,   -0.601159,    0.097693],   # O carbonílico (C=O)
        [-0.526732,    1.187406,   -0.238643],   # O do OH
        [ 1.418382,    0.112892,   -0.089695],   # H do OH (virado "para o lado" do C=O → cis)
    ]
)

cisCOOH.center(vacuum=6.0)  
cisCOOH.pbc = True

calc_relax = Vasp(

    directory = 'cisCOOHrelax',
    xc        = 'PBE',
    encut     = 450,
    ismear    = 0,
    sigma     = 0.05,
    ediff     = 1E-6, 
    ediffg    = -0.01,
    isif      = 2, 
    ibrion    = 2,
    nsw       = 100,
    nelm      = 100,
    lwave     = True,
    lcharg    = True,
    lvtot     = True,
    atoms     = cisCOOH

)   
calc_relax.calculate(cisCOOH)

E_cisCOOH = cisCOOH.get_potential_energy()

print(f'Optimized energy of cisCOOH: {E_cisCOOH:.3f} eV')

view(cisCOOH, viewer='x3d')

Optimized energy of cisCOOH: -23.933 eV


### $\text{cis-COOH}_{\text{(g)}}$: Vibration

In [120]:
calc_vib = Vasp(
    directory = 'cisCOOHvib',
    xc        = 'PBE',
    encut     = 450,
    ismear    = 0,
    sigma     = 0.05,
    ediff     = 1e-8,
    ibrion    = 6,           # finite differences with symmetry
    lreal     = False,
    lwave     = False, 
    lcharg    = False,
    atoms     = cisCOOH
)

calc_vib.calculate(cisCOOH)

cisCOOHvib = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = cisCOOHvib.get_frequencies()

print(f'Vibrational frequencies of cis-COOH (cm⁻¹):')
vib_energies_cooh = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energies_cooh.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

Vibrational frequencies of cis-COOH (cm⁻¹):
f_v: 552.207 cm⁻¹, E_v: 0.068 eV
f_v: 568.388 cm⁻¹, E_v: 0.070 eV
f_v: 1002.136 cm⁻¹, E_v: 0.124 eV
f_v: 1242.765 cm⁻¹, E_v: 0.154 eV
f_v: 1798.280 cm⁻¹, E_v: 0.223 eV
f_v: 3355.537 cm⁻¹, E_v: 0.416 eV


### Calculating enthalpy and Gibbs energy of reaction

$$
\text{CO}_{(\text{g})} + \text{HO}^{\bullet}_{(\text{g})} \longrightarrow \text{cis-HOOC}^{\bullet}_{(\text{g})}
$$

In [121]:

COthermo      = IdealGasThermo(vib_energies=vib_energies_co,
                        potentialenergy=E_CO, atoms=CO,
                        geometry='linear', symmetrynumber=1, spin=0)

OHthermo      = IdealGasThermo(vib_energies=vib_energies_oh,
                        potentialenergy=E_OH, atoms=OH,
                        geometry='linear', symmetrynumber=1, spin=0)

cisCOOHthermo = IdealGasThermo(vib_energies=vib_energies_cooh,
                        potentialenergy=E_cisCOOH, atoms=cisCOOH,
                        geometry='nonlinear', symmetrynumber=1, spin=0)

P = 101325. # Pa
T = 298.15  # K

G_CO      =      COthermo.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_CO      =      COthermo.get_enthalpy(temperature=T, verbose=False)

G_OH      =      OHthermo.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_OH      =      OHthermo.get_enthalpy(temperature=T, verbose=False)

G_cisCOOH = cisCOOHthermo.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_cisCOOH = cisCOOHthermo.get_enthalpy(temperature=T, verbose=False)

Grxn = - G_CO - G_OH + G_cisCOOH
Hrxn = - H_CO - H_OH + H_cisCOOH

print(f'CO(g) + HO*(g) ⟶ HOOC*(g) at {T} K and {P} Pa: \n')
print(f'ΔG = {Grxn:.3f} eV = {Grxn*96.485:.3f} kJ/mol')
print(f'ΔH = {Hrxn:.3f} eV = {Hrxn*96.485:.3f} kJ/mol')

CO(g) + HO*(g) ⟶ HOOC*(g) at 298.15 K and 101325.0 Pa: 

ΔG = -1.569 eV = -151.355 kJ/mol
ΔH = -1.951 eV = -188.205 kJ/mol


### $\text{Pt}_{\text{(s)}}$: Relaxation

In [122]:
Ptcrystal = FaceCenteredCubic(size=(1, 1, 1), symbol='Pt', latticeconstant=3.967, pbc=True)

calc_relax = Vasp(directory='Ptrelax',
                xc     = 'PBE',
                kpts   = [6, 6, 6],  # specifies k-points
                encut  = 450,
                ismear = 0, 
                sigma  = 0.05,
                ediff  = 1E-6, 
                ediffg = -0.01,
                isif   = 7, 
                ibrion = 2, 
                nsw    = 100, # ionic relaxation (ISIF=7: relax cell volume only)
                atoms  = Ptcrystal
                )
calc_relax.calculate(Ptcrystal) # demora por volta de 2 s

E_Pt = Ptcrystal.get_potential_energy()

a0_Pt = Ptcrystal.cell.cellpar()[0]

print(f'Optimized energy of Pt: {E_Pt:.3f} eV')
print(f"Lattice length: {a0_Pt:.3f} Å")

view(Ptcrystal, viewer='x3d')

Optimized energy of Pt: -24.458 eV
Lattice length: 3.967 Å


### $\text{Pt}(111)$ slab: Relaxation

In [123]:
# Superfície Pt111
Pt111slab = fcc111("Pt", size=(2,2,3), a=a0_Pt)
Pt111slab.center(vacuum=5.0, axis=2)
Pt111slab.pbc = True

calc_relax = Vasp(directory='Pt111slabrelax',
                xc     = 'PBE',
                kpts   = [6, 6, 1],  # specifies k-points
                encut  = 450,
                ismear = 0, 
                sigma  = 0.05,
                ediff  = 1E-6, 
                ediffg = -0.01,
                ibrion = -1, # just SCF (no ionic relaxation)
                atoms  = Pt111slab
                )
calc_relax.calculate(Pt111slab)

E_Pt111slab = Pt111slab.get_potential_energy()

print(f'Optimized energy of Pt111: {E_Pt111slab:.3f} eV')

view(Pt111slab, viewer='x3d')

Optimized energy of Pt111: -67.924 eV


### $\text{CO}_{\text{(ads)}}$: Relaxation

In [ ]:
COads = fcc111("Pt", size=(2,2,3), a=a0_Pt)
COads.center(vacuum=5.0, axis=2)
COads.pbc = True

CO = Atoms('CO',
            positions=[[6.        , 6.        , 5.99220197],
                       [6.        , 6.        , 7.13579803]] # geometria otimizada com PBE
)

add_adsorbate(COads,  adsorbate=CO, height=2.13, position = 'ontop', mol_index=0)

constraint = FixAtoms(mask=[atom.symbol=='Pt' for atom in COads])
COads.set_constraint([constraint])

calc_relax = Vasp(directory='COadsrelax',
                xc     = 'PBE',
                kpts   = [6, 6, 1],  # specifies k-points
                encut  = 450,
                ismear = 0, 
                sigma  = 0.05,
                ediff  = 1E-6, 
                ediffg = -0.01,
                isif   = 0, 
                ibrion = 2, 
                nsw    = 100, # ionic relaxation (ISIF=2: relax atoms position only)
                atoms  = COads
                )

calc_relax.calculate(COads) 

E_COads = COads.get_potential_energy() - E_Pt111slab

print(f'Optimized energy of CO in top position of Pt111: {E_COads:.3f} eV')

view(COads, viewer='x3d')

Optimized energy of CO in top position of Pt111: -84.337 eV


### $\text{CO}_{\text{(ads)}}$: Vibration

In [125]:
calc_vib = Vasp(
    directory='COadsvib',
    xc     = 'PBE',
    kpts   = [6, 6, 1],
    encut  = 450,
    ismear = 0, 
    sigma  = 0.05,
    ediff  = 1E-8,
    ibrion = 5,
    nfree  = 2, 
    nsw    = 1,           # finite differences with selective dynamics (Pt atoms fixed)
    lreal  = False,
    lwave  = False, 
    lcharg = False,
    atoms  = COads
)

calc_vib.calculate(COads)

COadsvib = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = COadsvib.get_frequencies()

print(f'Vibrational frequencies of CO on Pt111 (cm⁻¹):')

vib_energies_COads = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energies_COads.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

Vibrational frequencies of CO on Pt111 (cm⁻¹):
f_v: 372.455 cm⁻¹, E_v: 0.046 eV
f_v: 372.474 cm⁻¹, E_v: 0.046 eV
f_v: 467.995 cm⁻¹, E_v: 0.058 eV
f_v: 2059.789 cm⁻¹, E_v: 0.255 eV


### $\text{OH}_{\text{(ads)}}$: Relaxation

In [ ]:
OHads = fcc111("Pt", size=(2,2,3), a=a0_Pt)
OHads.center(vacuum=5.0, axis=2)
OHads.pbc = True

OH = Atoms('OH',
            positions=[[6.        , 6.        , 5.99220197],
                       [6.        , 6.        , 7.13579803]] # geometria otimizada com PBE
)

add_adsorbate(OHads,  adsorbate=OH, height=2.13, position = 'fcc', mol_index=0)

constraint = FixAtoms(mask=[atom.symbol=='Pt' for atom in OHads])
OHads.set_constraint([constraint])

calc_relax = Vasp(directory='OHadsrelax',
                xc     = 'PBE',
                kpts   = [6, 6, 1],  # specifies k-points
                encut  = 450,
                ismear = 0, 
                sigma  = 0.05,
                ediff  = 1E-6, 
                ediffg = -0.01,
                isif   = 0, 
                ibrion = 2, 
                nsw    = 100, # ionic relaxation (ISIF=2: relax atoms position only)
                atoms  = OHads
                )

calc_relax.calculate(OHads) 

E_OHads = OHads.get_potential_energy() - E_Pt111slab

print(f'Optimized energy of OH in top position of Pt111: {E_OHads:.3f} eV')

view(OHads, viewer='x3d')

Optimized energy of OH in top position of Pt111: -77.528 eV


### $\text{OH}_{\text{(ads)}}$: Vibration

In [127]:
calc_vib = Vasp(
    directory='OHadsvib',
    xc     = 'PBE',
    kpts   = [6, 6, 1],
    encut  = 450,
    ismear = 0, 
    sigma  = 0.05,
    ediff  = 1E-8,
    ibrion = 5,
    nfree  = 2, 
    nsw    = 1,           # finite differences with selective dynamics (Pt atoms fixed)
    lreal  = False,
    lwave  = False, 
    lcharg = False,
    atoms  = OHads
)

calc_vib.calculate(OHads)

OHadsvib = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = OHadsvib.get_frequencies()

print(f'Vibrational frequencies of OH on Pt111 (cm⁻¹):')

vib_energies_OHads = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energies_OHads.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

Vibrational frequencies of OH on Pt111 (cm⁻¹):
f_v: 320.526 cm⁻¹, E_v: 0.040 eV
f_v: 434.772 cm⁻¹, E_v: 0.054 eV
f_v: 434.891 cm⁻¹, E_v: 0.054 eV
f_v: 3618.322 cm⁻¹, E_v: 0.449 eV


### $\text{COOH}_{\text{(ads)}}$: Relaxation

In [ ]:
cisCOOHads = fcc111("Pt", size=(2,2,3), a=a0_Pt)
cisCOOHads.center(vacuum=5.0, axis=2)
cisCOOHads.pbc = True

cisCOOH = Atoms('COOH',
                positions=[
                            [-0.421859,    0.003009,    0.018584],   # C
                            [ 0.773521,   -0.601159,    0.097693],   # O carbonílico (C=O)
                            [-0.526732,    1.187406,   -0.238643],   # O do OH
                            [ 1.418382,    0.112892,   -0.089695],   # H do OH (virado "para o lado" do C=O → cis)
                        ] # geometria otimizada com PBE
)


cisCOOH.rotate('x', 100.)
cisCOOH.rotate('y', -30.)
cisCOOH.rotate('z', 30.)  

add_adsorbate(cisCOOHads,  adsorbate=cisCOOH, height=2.13, position = 'ontop', mol_index=0)

constraint = FixAtoms(mask=[atom.symbol=='Pt' for atom in cisCOOHads])
cisCOOHads.set_constraint([constraint])

calc_relax = Vasp(directory='cisCOOHadsrelax',
                xc     = 'PBE',
                kpts   = [6, 6, 1],  # specifies k-points
                encut  = 450,
                ismear = 0, 
                sigma  = 0.05,
                ediff  = 1E-6, 
                ediffg = -0.01,
                isif   = 0, 
                ibrion = 2, 
                nsw    = 100, # ionic relaxation (ISIF=2: relax atoms position only)
                atoms  = cisCOOHads
                )

calc_relax.calculate(cisCOOHads) 

E_cisCOOHads = cisCOOHads.get_potential_energy() - E_Pt111slab

print(f'Optimized energy of cisCOOH in top position of Pt111: {E_cisCOOHads:.3f} eV')

view(cisCOOHads, viewer='x3d')

### $\text{COOH}_{\text{(ads)}}$: Vibration

In [129]:
calc_vib = Vasp(
    directory='cisCOOHadsvib',
    xc     = 'PBE',
    kpts   = [6, 6, 1],
    encut  = 450,
    ismear = 0, 
    sigma  = 0.05,
    ediff  = 1E-8,
    ibrion = 5,
    nfree  = 2, 
    nsw    = 1,           # finite differences with selective dynamics (Pt atoms fixed)
    lreal  = False,
    lwave  = False, 
    lcharg = False,
    atoms  = cisCOOHads
)

calc_vib.calculate(cisCOOHads)

cisCOOHadsvib = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = cisCOOHadsvib.get_frequencies()

print(f'Vibrational frequencies of cisCOOH on Pt111 (cm⁻¹):')

vib_energies_cisCOOHads = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energies_OHads.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

Vibrational frequencies of cisCOOH on Pt111 (cm⁻¹):
f_v: 271.782 cm⁻¹, E_v: 0.034 eV
f_v: 283.622 cm⁻¹, E_v: 0.035 eV
f_v: 461.959 cm⁻¹, E_v: 0.057 eV
f_v: 619.773 cm⁻¹, E_v: 0.077 eV
f_v: 640.039 cm⁻¹, E_v: 0.079 eV
f_v: 1030.962 cm⁻¹, E_v: 0.128 eV
f_v: 1232.688 cm⁻¹, E_v: 0.153 eV
f_v: 1669.402 cm⁻¹, E_v: 0.207 eV
f_v: 3567.267 cm⁻¹, E_v: 0.442 eV


### Calculating enthalpy and Gibbs energy of reaction

$$
\text{CO}_{(\text{ads})} + \text{HO}^{\bullet}_{(\text{ads})} \longrightarrow \text{cis-HOOC}^{\bullet}_{(\text{ads})}
$$

In [ ]:
COadsthermo      = IdealGasThermo(vib_energies=vib_energies_COads,
                        potentialenergy=E_COads, atoms=COads,
                        geometry='nonlinear', symmetrynumber=2, spin=0)

OHadsthermo      = IdealGasThermo(vib_energies=vib_energies_OHads,
                        potentialenergy=E_OHads, atoms=OHads,
                        geometry='nonlinear', symmetrynumber=2, spin=0)

cisCOOHadsthermo = IdealGasThermo(vib_energies=vib_energies_cisCOOHads,
                        potentialenergy=E_cisCOOHads, atoms=cisCOOHads,
                        geometry='nonlinear', symmetrynumber=2, spin=0)

P = 101325. # Pa
T = 298.15  # K

G_COads      =      COadsthermo.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_COads      =      COadsthermo.get_enthalpy(temperature=T, verbose=False)

G_OHads      =      OHadsthermo.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_OHads      =      OHadsthermo.get_enthalpy(temperature=T, verbose=False)

G_cisCOOHads = cisCOOHadsthermo.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_cisCOOHads = cisCOOHadsthermo.get_enthalpy(temperature=T, verbose=False)

Grxnads = - G_COads - G_OHads + G_cisCOOHads
Hrxnads = - H_COads - H_OHads + H_cisCOOHads

print(f'CO(ads) + HO*(ads) ⟶ HOOC*(ads) at {T} K and {P} Pa: \n')
print(f'ΔG = {Grxnads:.3f} eV = {Grxnads*96.485:.3f} kJ/mol')
print(f'ΔH = {Hrxnads:.3f} eV = {Hrxnads*96.485:.3f} kJ/mol')

CO(ads) + HO*(ads) ⟶ HOOC*(ads) at 298.15 K and 101325.0 Pa: 

ΔG = 67.280 eV = 6491.485 kJ/mol
ΔH = 66.008 eV = 6368.814 kJ/mol
